## Load parameters

In [1]:
parameter_file = '../parameters/STAR_get_bam.yaml'

import yaml
import sys
import glob
import os
from os.path import join, basename
import pandas as pd

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

print(doc)

sys.path.append(doc["NGS_folder"])
sys.path.append(os.path.join(doc["NGS_folder"],'Modules'))
from f01_file_process import *
p = dic2obj(**doc)
p

# data_folder = doc['data_folder']
# tissues = doc['tissues'].split(',')
# sys.path.append(doc['pipeline_path'])
# ref_fa = doc['ref_fa']
# #annotation=doc['annotation_gtf']
# annotation=doc['annotation']
# from Homer import *


{'contact': 'ishamie14@gmail.com', 'RawDataPath': '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/', 'NGS_folder': '/home/isshamie/software/NGS-Pipeline', 'thread': 8, 'ref_fa': '/data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna', 'gff': '/data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3', 'annotation': '/data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3', 'gtf': '/data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf', 'QC': False, 'trim_reads': False, 'trimmomatic_path': '/home/shangzhong/Installation/Trimmomatic-0.32/Trimmomatic-0.33/trimmomatic-0.33.jar', 'trim_jobs_per_batch': 6, 'adapter': '', 'picard': '/home/shangzhong/Installation/picard-tools-1.141/picard.jar', 'star_jobs_per_batch': 2, 'star_index_path': '/data/isshamie/genome/ncbi_anno_103/CriGri-PICR_STAR_db', 'STAR_index_path': '/data/isshamie/genome/ncbi_anno_103/CriGri-PICR_STAR_db', 'star_pass': 2, 'star_params': [], 'htseq_anno_source': 'ncbi', '

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  # This is added back by InteractiveShellApp.init_path()


In [2]:
file_path = p.RawDataPath
thread = p.thread
QC = p.QC
# all parameter
ref_fa = p.ref_fa
annotation = p.gff
gtf_anno = p.gtf
# trimmomatic parameter
trim = p.trim_reads
trimmomatic = p.trimmomatic_path
trim_batch = p.trim_jobs_per_batch
adapter = p.adapter
# star parameter
star_batch = p.star_jobs_per_batch
db_path = p.STAR_index_path
# htseq parameter
htseq_anno_source = p.htseq_anno_source
strand = p.strand_specific
id_name = p.id_name

contact = p.contact

## Make directories

In [3]:
qc_dir = join(file_path,"QC")
multiqc_dir = join(file_path,"multiqc")


trim_qc_dir = join(file_path,"trim_QC")
trim_multiqc_dir = join(file_path,"trim_multiqc")


bam_dir = join(file_path, "sortBam")

ht_dir = join(file_path, 'htseq')

tag_dir = join(file_path, 'tags')

RNA_SeQC = join(file_path,'RNA_SeQC')

## Running shangzhong's pipeline RNAseq_count.py
(Will generate bam and htseq count data

The star command is this:  
```STAR --genomeDir /data/isshamie/genome/ncbi_anno_103/CriGri-PICR_STAR_db --readFilesCommand zcat --readFilesIn NL11_Brain7-1_S11_L006_R1_001.fastq.gz NL11_Brain7-1_S11_L006_R2_001.fastq.gz --runThreadN 4 --outFileNamePrefix bam/NL11_Brain7-1_S11_L006_R1_001.fastq.bam --outSAMstrandField intronMotif --outFilterIntronMotifs RemoveNoncanonical --outSAMtype BAM Unsorted --outSAMunmapped Within --sjdbGTFfile /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3 --sjdbGTFtagExonParentTranscript Parent```

## Next, need to run QC on RNA 

## Picard QC stats

In [4]:
import inspect
import textwrap

In [5]:
# cmd = """samtools index ${md_bam_file}"""

# for pair in list_fq_files(p.RawDataPath):
#     sample_id = basename(pair[0]).replace('.fastq',"").replace(".gz","")
#     sample_id = "_".join(sample_id.split("_")[1:3])
#     print(sample_id)
#     md_bam_file = join(bam_dir,sample_id)
    
#     javaPath="/home/vahid/.conda/pkgs/java-1.7.0-openjdk-cos6-x86_64-1.7.0.131-h06d78d4_0/x86_64-conda_cos6-linux-gnu/sysroot/usr/lib/jvm/java-1.7.0-openjdk-1.7.0.131.x86_64/jre/bin/" 
#     cmd = f"""
#     {javaPath}java -jar /data/vahid/RNAseqPipeLine/RNA-SeQC_1.1.9/RNA-SeQC.jar -n 1000 \
#         -s {sample_id},{md_bam_file},{sample_id} \
#         -t {gtf_anno} \
#         -r {ref_fa} \
#         -noDoC \
#         -strictMode \
#         -o {RNA_SeQC} \
#         -gatkFlags --allow_potentially_misencoded_quality_scores \
#         -singleEnd no
#         """

#     print(inspect.cleandoc(cmd))
#     os.system(cmd)

In [5]:
if not os.path.exists(qc_dir):
    os.mkdir(qc_dir)

if not os.path.exists(multiqc_dir):
    os.mkdir(multiqc_dir)
    
if not os.path.exists(trim_qc_dir):
    os.mkdir(trim_qc_dir)

if not os.path.exists(trim_multiqc_dir):
    os.mkdir(trim_multiqc_dir)
    
if not os.path.exists(bam_dir):
    os.mkdir(bam_dir)
    
if not os.path.exists(ht_dir):
    os.mkdir(ht_dir)
    
if not os.path.exists(tag_dir):
    os.mkdir(tag_dir)
    
if not os.path.exists(RNA_SeQC):
    os.mkdir(RNA_SeQC)

## Make the tag directory

In [6]:
import os
import glob
homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer

/data2/resources/software/homer/v4.11_10242019/bin//homer


In [8]:
#makeTagDirectory TagDir/ alignment.sam -read2

for curr in glob.glob(bam_dir+'/*.bam'):
    print(curr)
    sample_id = basename(curr).replace('.fastq',"").replace(".gz","")
    sample_id = "_".join(sample_id.split("_")[1:3])
    sample_id = sample_id.split("-")[0]
    print(sample_id)
    md_bam_file = join(bam_dir,sample_id)
    

    cmd = f'makeTagDirectory {join(tag_dir,sample_id) } {curr} -read2 -single' #'-sspe -unique'
    print(cmd)
    os.system(cmd)

/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL10_Misc_S10_L006_R1_001.fastq.sorted.bam
Misc_S10
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Misc_S10 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL10_Misc_S10_L006_R1_001.fastq.sorted.bam -read2 -single
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL4_Lung_S4_L006_R1_001.fastq.sort.bam
Lung_S4
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Lung_S4 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL4_Lung_S4_L006_R1_001.fastq.sort.bam -read2 -single
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL7_Brain2_S7_L006_R1_001.fastq.sort.bam
Brain2_S7
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Brain2_S7 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL7_Brain2_S7_L006_R1_001.fastq.sort.bam -read2 -single
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL6_Heart_S6_L006_R1_001.fastq.so

## Fragment length specified

In [9]:
#makeTagDirectory TagDir/ alignment.sam -read2

for curr in glob.glob(bam_dir+'/*.bam'):
    print(curr)
    sample_id = basename(curr).replace('.fastq',"").replace(".gz","")
    sample_id = "_".join(sample_id.split("_")[1:3])
    sample_id = sample_id.split("-")[0]
    print(sample_id)
    md_bam_file = join(bam_dir,sample_id)
    
    cmd = f'makeTagDirectory {join(tag_dir,sample_id+"_frag150")} {curr} -read2 -fragLength 150 -single' 
    print(cmd)
    os.system(cmd)

/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL10_Misc_S10_L006_R1_001.fastq.sorted.bam
Misc_S10
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Misc_S10_frag150 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL10_Misc_S10_L006_R1_001.fastq.sorted.bam -read2 -fragLength 150 -single
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL4_Lung_S4_L006_R1_001.fastq.sort.bam
Lung_S4
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Lung_S4_frag150 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL4_Lung_S4_L006_R1_001.fastq.sort.bam -read2 -fragLength 150 -single
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL7_Brain2_S7_L006_R1_001.fastq.sort.bam
Brain2_S7
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Brain2_S7_frag150 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL7_Brain2_S7_L006_R1_001.fastq.sort.bam -read2 -fragLength 150 -single
/data/isshamie/TSS/

## read1 used

In [7]:
#makeTagDirectory TagDir/ alignment.sam -read2
tag_dir_read1 = join(tag_dir+"_read1")
if not os.path.exists(tag_dir_read1):
    os.mkdir(tag_dir_read1)

for curr in glob.glob(bam_dir+'/*.bam'):
    print(curr)
    sample_id = basename(curr).replace('.fastq',"").replace(".gz","")
    sample_id = "_".join(sample_id.split("_")[1:3])
    sample_id = sample_id.split("-")[0]
    print(sample_id)
    md_bam_file = join(bam_dir,sample_id)
    

    cmd = f'makeTagDirectory {join(tag_dir_read1,sample_id) } {curr} -read1 -single' #'-sspe -unique'
    print(cmd)
    os.system(cmd)

/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL10_Misc_S10_L006_R1_001.fastq.sorted.bam
Misc_S10
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags_read1/Misc_S10 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL10_Misc_S10_L006_R1_001.fastq.sorted.bam -read1 -single
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL4_Lung_S4_L006_R1_001.fastq.sort.bam
Lung_S4
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags_read1/Lung_S4 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL4_Lung_S4_L006_R1_001.fastq.sort.bam -read1 -single
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL7_Brain2_S7_L006_R1_001.fastq.sort.bam
Brain2_S7
makeTagDirectory /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags_read1/Brain2_S7 /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL7_Brain2_S7_L006_R1_001.fastq.sort.bam -read1 -single
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL6_Heart_S6_L0

In [7]:
# os.chdir(p.RawDataPath)
# for pair in list_fq_files(p.RawDataPath):
#     sample_name = basename(pair[0]).replace('.fastq',"").replace(".gz","")
#     sample_name = "_".join(sample_name.split("_")[1:3])
#     print(sample_name)
#     cmd =   f'{join(p.NGS_folder,"RNAseq_STARpipeline.sh")} -f1 {pair[0]} -f2 {pair[1]} -sid {sample_name} '
#     cmd = cmd + f' -starIndx {p.STAR_index_path} -BAM {join(bam_dir,sample_name)} -refFasta {p.ref_fa} -GFF {p.gff}'
#     cmd = cmd + f' -RSEM /data/vahid/RNAseqPipeLine/rsemRef/rsem_reference'
#     print(cmd)
#    #os.system(cmd)


Pancreas_S1
/home/isshamie/software/NGS-Pipeline/RNAseq_STARpipeline.sh -f1 NL1_Pancreas_S1_L006_R1_001.fastq.gz -f2 NL1_Pancreas_S1_L006_R2_001.fastq.gz -sid Pancreas_S1  -starIndx /data/isshamie/genome/ncbi_anno_103/CriGri-PICR_STAR_db -BAM /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/bam/Pancreas_S1 -refFasta /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -GFF /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3 -RSEM /data/vahid/RNAseqPipeLine/rsemRef/rsem_reference
ReproductiveTract_S2
/home/isshamie/software/NGS-Pipeline/RNAseq_STARpipeline.sh -f1 NL2_ReproductiveTract_S2_L006_R1_001.fastq.gz -f2 NL2_ReproductiveTract_S2_L006_R2_001.fastq.gz -sid ReproductiveTract_S2  -starIndx /data/isshamie/genome/ncbi_anno_103/CriGri-PICR_STAR_db -BAM /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/bam/ReproductiveTract_S2 -refFasta /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -GFF /data/isshamie/genome/ncbi_anno_